In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import os
import re
%matplotlib inline

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import jellyfish

# import torch
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('stsb-roberta-large')

# Imported the inaugural corpus from nltk.corpus
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import gensim
from gensim.utils import simple_preprocess
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import num2words
from num2words import num2words
import time
# from time import time

pd.options.display.max_colwidth = 10000

In [2]:
# pip uninstall tensorflow --y

In [3]:
masterkey_pdtdesc = pd.read_csv('~/Downloads/mapped_unique_masterkey_product_desc.csv', engine='python')

In [4]:
len(masterkey_pdtdesc)

53741

In [5]:
invoice_lessthan10_UPC_desc = pd.read_csv('~/Downloads/lessthan10_UPC_desc.csv', engine='python')

In [6]:
masterkey_pdtdesc.sort_values(by = 'PRODUCT_DESCRIPTION', inplace = True)

In [7]:
invoice_lessthan10_UPC_desc.sort_values(by = 'INVOICE_PACKAGE_DESCRIPTION', inplace = True)

In [8]:
len(invoice_lessthan10_UPC_desc)

630036

In [22]:
subset_invoice = invoice_lessthan10_UPC_desc[:10]

In [23]:
subset_invoice.head(100)

,Unnamed: 0,INVOICE_PACKAGE_DESCRIPTION
173592,173592,MAMAMANGO 750ML
219482,219482,SAY WHEN DOUBLE IPA 4 PACK
222877,222877,10% MARK UP
383737,383737,12 PACK BUD LIGHT CANS
628030,628030,16OZ H.PL. 20/50
67993,67993,2009 CH. ST. SATURNIN
342852,342852,"2010 CH. QUEYRON PINDEFLEURS, ST. EMILION G.C."
263733,263733,2011 CH. QUEYRON PINDEFLEURS
129243,129243,"2013 RION, CHAMBOLLE-MUSIGNY 'LES ECHEZEAUX'"
287214,287214,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML"


In [24]:
subset_master = masterkey_pdtdesc[:100]

In [12]:
subset_master.columns

Index(['Unnamed: 0', 'PRODUCT_MASTER_CATALOG_KEY', 'PRODUCT_DESCRIPTION'], dtype='object')

In [13]:
start = time.time()

In [15]:
ct = 0
ratio = []
n = 0
for i in subset_invoice.index:
    ct += 1
    for j in subset_master.index:
        
        ratio.append([subset_master.loc[j,"PRODUCT_MASTER_CATALOG_KEY"],subset_invoice.loc[i,"INVOICE_PACKAGE_DESCRIPTION"],subset_master.loc[j,"PRODUCT_DESCRIPTION"],fuzz.partial_ratio(subset_invoice.loc[i,"INVOICE_PACKAGE_DESCRIPTION"],subset_master.loc[j,"PRODUCT_DESCRIPTION"])])

        ratiox = pd.DataFrame(ratio, columns=('PRODUCT_MASTER_CATALOG_KEY','INVOICE_PACKAGE_DESCRIPTION','PRODUCT_DESCRIPTION','ratio_score')).sort_values(['INVOICE_PACKAGE_DESCRIPTION','ratio_score'],ascending=False).groupby("INVOICE_PACKAGE_DESCRIPTION").head(10).reset_index()
        export_file = ratiox
    print(ct)
    if ct == 10000:
        n += 1
        print("---->", n)
        file1 = open("Unmatched_fuzzymatch{}_top10_ratios.csv".format(n), "a") 
        export_file.to_csv(file1, header=False)
        file1.close()
        ratio = []
        del ratiox
        ct =0

    
    

KeyboardInterrupt: 

In [81]:
def logic(invoice_desc,ct,n):

    l =[]
    l.append(invoice_desc)

    compare = pd.MultiIndex.from_product([l, subset_master['PRODUCT_DESCRIPTION']]).to_series()
    df_compare = compare.reset_index()
    df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
    df_compare.dropna(axis='rows',inplace = True)
#calling partial model for score     
    df_compare['partial'], df_compare['set_ratio'] = df_compare['concat'].apply(partial)
    df_compare.rename(columns={ df_compare.columns[0]: "INVOICE_PACKAGE_DESCRIPTION" }, inplace = True)
    res = df_compare.sort_values('partial',ascending=False).head(10)
    
    if ct == 2:
        print("--->",n)
        n += 1
        export_file = res[['INVOICE_PACKAGE_DESCRIPTION','PRODUCT_DESCRIPTION','partial','set_ratio']]
        file1 = open("N_unmatched_fuzzymatch_top10_ratios{}.csv".format(n), "a") 
        export_file.to_csv(file1, header=False)
        file1.close()
        del df_compare
        del res
        ct = 0
        

In [82]:
def partial(tup):
    return pd.Series([fuzz.partial_ratio(*tup), fuzz.token_set_ratio(*tup)],['partial','set_ratio'])

In [83]:

ratio = []
n = 0
for i in subset_invoice.index:
    ct = 0
    ct += 1
    x = subset_invoice.loc[i, "INVOICE_PACKAGE_DESCRIPTION"]
    logic(x,ct,n)

        

KeyError: 'partial'

In [ ]:
end = time.time()
print("time taken for 100k:" (end-start))

In [23]:
subset_master["joined"] = subset_master['PRODUCT_MASTER_CATALOG_KEY'].map(str) + " |" + subset_master['PRODUCT_DESCRIPTION'] 

In [24]:
rgx_list = [r'\s+\d+\s+CT\s+\d+\s+OZ', r'\d+\s+OZ', r'\s+\d+/\d+',r'\s+\d+OZ', r'CANS', r'CAN', r'OZ', r'BOX' , r'IN',r'PK', r'CN', r'\d+', r'CASE',
           r'\d+L/d+\s+PET',r'\.',r'\.L',r'NR', r'/',r'O', r'()',r'//','LTR', r'PET',r'ML',r'\?',r'BBL']

In [25]:
import re
def remove_pattern(rgx_list, text):
    for r in rgx_list:
        
        text = re.sub(r, '', text )
        
    return text


In [26]:
subset_invoice['clean_invoice_desc'] = subset_invoice.apply(lambda x: remove_pattern(rgx_list,x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 
subset_master['clean_product_desc'] = subset_master.apply(lambda x: remove_pattern(rgx_list,x.PRODUCT_DESCRIPTION), axis=1) 



In [54]:
subset_invoice['invoice_embeddings'] = subset_invoice.apply(lambda x:model.encode(x.INVOICE_PACKAGE_DESCRIPTION,convert_to_tensor=True), axis=1)
                                                           

In [94]:
print(type(subset_invoice['invoice_embeddings']))

<class 'pandas.core.series.Series'>


In [ ]:
start = 

In [191]:
subset_master['product_embeddings'] = subset_master.apply(lambda x:model.encode(x.PRODUCT_DESCRIPTION,convert_to_tensor=True), axis=1)


In [118]:
def partial(tup):
#     l.append(tup)
    #Deviding the joined string for fuzzymatch
#     sep = ''
#     out = sep.join(tup[1])
#     n = out.split('|')
#     product_soundex = jellyfish.soundex(n[1])
#     invoice_soundex = jellyfish.soundex(tup[0])
#     product_soundex = jellyfish.soundex(tup[1])
    
    return pd.Series([fuzz.partial_ratio(*tup), fuzz.token_set_ratio(*tup)],['partial','set_ratio'])

In [35]:
def logic(invoice_desc):
    l =[]
    l.append(invoice_desc)
    print(len(subset_master['PRODUCT_DESCRIPTION']))
    compare = pd.MultiIndex.from_product([l, subset_master['PRODUCT_DESCRIPTION']]).to_series()
    df_compare = compare.reset_index()
    print(type(df_compare))
    df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
#calling partial model for score     

    df_compare = compare.apply(partial)
    df_compare.reset_index()
    print(df_compare.columns)
    df_compare.dropna(axis='rows',inplace = True)
    
    df_compare.rename(columns={ df_compare.columns[0]: "INVOICE_PACKAGE_DESCRIPTION" }, inplace = True)
    res = df_compare.sort_values('partial',ascending=False).head(10)
    export_file = res[['INVOICE_PACKAGE_DESCRIPTION','PRODUCT_DESCRIPTION','partial','set_ratio']]
    file1 = open("unmatched_fuzzymatch_top10_ratios4.csv", "a") 
    export_file.to_csv(file1, header=False)
    file1.close()

In [36]:
for i in subset_invoice.index:
    x = subset_invoice.loc[i, "INVOICE_PACKAGE_DESCRIPTION"]

    logic(x)

100
<class 'pandas.core.frame.DataFrame'>


IndexError: list index out of range

In [95]:
def partial2(tup):
#     l.append(tup)
    #Deviding the joined string for fuzzymatch
#     sep = ''
#     out = sep.join(tup[1])
#     n = out.split('|')
#     product_soundex = jellyfish.soundex(n[1])
#     invoice_soundex = jellyfish.soundex(tup[0])
#     product_soundex = jellyfish.soundex(tup[1])
#     r1 = model.encode(tup[0])
#     r2 = model.encode(tup[1])
    return pd.Series([util.pytorch_cos_sim(tup[0],tup[1])],['cosine'])

In [211]:

def logic2(invoice_desc):
#     l = []
#     l.append(invoice_desc)
#     print(type(subset_master['product_embeddings']))
    ratio = []  
    print(invoice_desc)
    for j in subset_master.index:
        ratio.append([subset_invoice.loc[i,"INVOICE_PACKAGE_DESCRIPTION"],subset_master.loc[j,"PRODUCT_DESCRIPTION"],util.pytorch_cos_sim(invoice_desc,subset_master.loc[j,"product_embeddings"])])
#     compare = pd.MultiIndex.from_product([invoice_desc ,subset_master['product_embeddings']]).to_series()
    
    
    ratiox = pd.DataFrame(ratio, columns=('INVOICE_PACKAGE_DESCRIPTION','PRODUCT_DESCRIPTION','ratio_score')).sort_values('ratio_score',ascending=False).reset_index()
    export_file = ratiox
    file1 = open("sentencesimilaity3_top10_ratios.csv", "a") 
    export_file.to_csv(file1, header=False)
    file1.close()
#     df_compare = compare.reset_index()
#     df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)
#     df_compare.dropna(axis='rows',inplace = True)
# #calling partial model for score     
    

#     df_compare['cosine'] = df_compare['concat'].apply(partial2)
#     df_compare.rename(columns={ df_compare.columns[0]: "invoice_desc" }, inplace = True)
#     res = df_compare.sort_values('cosine',ascending=False).head(15)
#     export_file = res[['invoice_desc','product_embeddings','cosine']]
#     file1 = open("sentencesimilaity_top10_ratios.csv", "a") 
#     export_file.to_csv(file1, header=False)
#     file1.close()

In [212]:
# ratio = []
# partial_ratio = []
# token_sort_ratio = []
# token_set_ratio = []
# for i in subset_invoice.index:
#     query = subset_invoice.loc[i,"invoice_embeddings"]
#     for j in subset_master.index:
#         ratio.append([util.pytorch_cos_sim(query,subset_master.loc[j,"product_embeddings"])])
    

In [213]:
start = time.time()

In [214]:
# logic(" 12 PACK BUD LIGHT CANS")

In [215]:
for i in subset_invoice.index:
    x = subset_invoice.loc[i, "invoice_embeddings"]

    logic2(x)

tensor([0.5045, 0.5136, 0.1209,  ..., 0.8500, 0.7067, 0.7955])
tensor([ 0.5480,  0.9498, -0.1607,  ..., -0.5912,  0.4417,  0.0266])
tensor([-1.0799,  0.3358, -0.5479,  ..., -0.3734,  0.0256, -0.5663])
tensor([-1.3009,  1.1014,  0.7082,  ..., -0.5337, -0.8171, -1.4519])
tensor([ 0.4064,  0.5794,  0.3497,  ...,  0.1318,  1.3028, -0.0137])
tensor([ 0.3476, -0.2565, -1.1898,  ..., -0.9758, -0.0598, -0.4876])
tensor([ 0.7317,  0.2887, -0.6703,  ..., -0.9365,  0.2918, -0.0604])
tensor([-0.3578,  0.3998, -0.3524,  ..., -1.3774, -1.0008, -0.1362])
tensor([ 0.2213,  0.4691,  0.3013,  ...,  0.5092, -0.8497, -0.5028])
tensor([ 0.7710,  0.7617,  0.1080,  ...,  0.9747, -0.1437, -0.4238])


In [187]:
ratiox

NameError: name 'ratiox' is not defined

In [108]:
end = time.time()
print("dsdsdsd", (end-start))

dsdsdsd 1.7544264793395996


In [ ]:
 
# subset_invoice['INVOICE_SOUNDEX'] = subset_invoice.apply(lambda x: jellyfish.soundex(x.INVOICE_PACKAGE_DESCRIPTION), axis=1) 


In [ ]:
fuzz.partial_ratio(" 12 PACK BUD LIGHT CANS","BUD LIGHT ORANGE BEER 24 CT 12 OZ LONG NECK BTL CRTN 288 OZ")



In [ ]:
fuzz.partial_ratio(" 12 PACK BUD LIGHT CANS" ,"RIVER NORTH MIXED 12 PACK - OZ CANS")

In [ ]:
subset_master

In [ ]:
# subset_master.columns

In [20]:
subset_master["joined"] = subset_master['PRODUCT_MASTER_CATALOG_KEY'].map(str) + " |" + subset_master['PRODUCT_DESCRIPTION'] 

In [25]:
compare = pd.MultiIndex.from_product([subset_invoice['INVOICE_PACKAGE_DESCRIPTION'],
                                      subset_master['joined'],
                                     ]).to_series()

In [26]:
# pd.DataFrame(index = compare).reset_index()
df_compare = compare.reset_index()

In [27]:
df_compare.columns

Index(['INVOICE_PACKAGE_DESCRIPTION', 'joined', 0], dtype='object')

In [101]:
df_compare.rename(columns={ df_compare.columns[2]: "concat" }, inplace = True)


In [102]:
df_compare

,INVOICE_PACKAGE_DESCRIPTION,joined,concat
0,MAMAMANGO 750ML,340646 |REDDS WICKED APPLE ALE BEER 1 CT 24 OZ CAN 24 OZ,"( MAMAMANGO 750ML, 340646 |REDDS WICKED APPLE ALE BEER 1 CT 24 OZ CAN 24 OZ)"
1,MAMAMANGO 750ML,366058 |KING ESTATE WILLAMETTE VALLEY SELECT PINOT GRIS 12/750ML,"( MAMAMANGO 750ML, 366058 |KING ESTATE WILLAMETTE VALLEY SELECT PINOT GRIS 12/750ML)"
2,MAMAMANGO 750ML,345460 |NEW BELGIUM VOODOO RANGER JUICY HAZE IPA BEER 6 CT 12 OZ LONG NECK BTL CRTN 72 OZ,"( MAMAMANGO 750ML, 345460 |NEW BELGIUM VOODOO RANGER JUICY HAZE IPA BEER 6 CT 12 OZ LONG NECK BTL CRTN 72 OZ)"
3,MAMAMANGO 750ML,341220 |COORS LIGHT BEER 15 CT 16 OZ ALUMINUM BOTTL IN BX 240 OZ,"( MAMAMANGO 750ML, 341220 |COORS LIGHT BEER 15 CT 16 OZ ALUMINUM BOTTL IN BX 240 OZ)"
4,MAMAMANGO 750ML,361732 |SUTTER HOME VNZ ITL PIN GRIGO 1.5 LTR,"( MAMAMANGO 750ML, 361732 |SUTTER HOME VNZ ITL PIN GRIGO 1.5 LTR)"
...,...,...,...
995,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",75349 |KRAGLE FREE WILL BREWING CO 16FL. OZ. CAN,"( 2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML, 75349 |KRAGLE FREE WILL BREWING CO 16FL. OZ. CAN)"
996,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",343233 |BUD LIGHT BEER 12 CT 40 OZ GLASS BOTTLE 480 OZ,"( 2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML, 343233 |BUD LIGHT BEER 12 CT 40 OZ GLASS BOTTLE 480 OZ)"
997,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",366047 |THE FEDERALIST LODI CABERNET SAUVIGNON 750ML,"( 2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML, 366047 |THE FEDERALIST LODI CABERNET SAUVIGNON 750ML)"
998,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",348575 |GUINNESS EXTRA STOUT BEER 24 CT 12 OZ GLASS BOTTLE IN BOX 288 OZ,"( 2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML, 348575 |GUINNESS EXTRA STOUT BEER 24 CT 12 OZ GLASS BOTTLE IN BOX 288 OZ)"


In [103]:
# # # df_compare.drop(df_compare.index[[253251,253252]],inplace = True )
# df_compare = df_compare.drop(df_compare[df_compare.joined == "253251 | \\M/ WARPIG ATHENS IPA"].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172269 |© 3 2012 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172270 |© 3 2013 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172262 |© 3 2013 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172261 |© 3 2012 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172263 |© 3 2014 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172271 |© 3 2014 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172272 |© 3 2015 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172264 |© 3 2015 CABERNET SAUVIGNON'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172273 |© 3 2015 WHITE BLEND'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172265 |© 3 2015 WHITE BLEND'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172266 |© 3 2016 SAUVIGNON BLANC'].index)
# df_compare = df_compare.drop(df_compare[ df_compare.joined == '172274 |© 3 2016 SAUVIGNON BLANC'].index)
# # df_compare = df_compare.drop(df_compare[ df_compare.joined == '© 3 2016 SAUVIGNON BLANC'].index)
# # df_compare = df_compare.drop(df_compare[ df_compare.joined == '© 3 2016 SAUVIGNON BLANC'].index)
# # df_compare = df_compare.drop(df_compare[ df_compare.joined == '© 3 2016 SAUVIGNON BLANC'].index)




In [104]:
df_compare.dropna(axis='rows',inplace = True)

In [105]:
len(df_compare)

1000

In [106]:
start = time.time()

In [29]:
def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup),
                     fuzz.partial_ratio(*tup)],
                     ['ratio', 'token','partial'])
l =[]


def partial(tup):
#     l.append(tup)
    invoice_soundex = jellyfish.soundex(tup[0])
    #Deviding the joined string for fuzzymatch
    sep = ''
    out = sep.join(tup[1])
    n = out.split('|')

    product_soundex = jellyfish.soundex(n[1])

    return pd.Series([fuzz.partial_ratio(invoice_soundex,product_soundex)],['partial'])

In [30]:
compare

INVOICE_PACKAGE_DESCRIPTION                                                 joined                                                   
  MAMAMANGO 750ML                                                           122436 | CBA EXPLORERS VARIETY PACK 2/12 LNNR                                                                                     (  MAMAMANGO 750ML, 122436 | CBA EXPLORERS VARIETY PACK 2/12 LNNR)
                                                                            122449 | KONA BIG KAHUNA VARIETY 24/12 NR                                                                                             (  MAMAMANGO 750ML, 122449 | KONA BIG KAHUNA VARIETY 24/12 NR)
                                                                            122468 | KONA BIG WAVE 1/6 KEG                                                                                                                   (  MAMAMANGO 750ML, 122468 | KONA BIG WAVE 1/6 KEG)
                                               

In [31]:
x = compare.apply(metrics)

In [32]:
x.columns

Index(['ratio', 'token', 'partial'], dtype='object')

In [33]:
x.reset_index()

,INVOICE_PACKAGE_DESCRIPTION,joined,ratio,token,partial
0,MAMAMANGO 750ML,122436 | CBA EXPLORERS VARIETY PACK 2/12 LNNR,19,14,29
1,MAMAMANGO 750ML,122449 | KONA BIG KAHUNA VARIETY 24/12 NR,21,19,29
2,MAMAMANGO 750ML,122468 | KONA BIG WAVE 1/6 KEG,21,14,29
3,MAMAMANGO 750ML,122463 | KONA BIG WAVE 4/6/12 LNNR,20,13,29
4,MAMAMANGO 750ML,122496 | KONA BIG WAVE 6/4 16OZ CAN,23,17,31
...,...,...,...,...,...
995,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",132329 |10 BARREL 1/2,17,16,29
996,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",132328 |10 BARREL 1/6,17,18,29
997,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",132311 |10 BARREL APOCALYPSE 12/12 12OZ BOTTLE LOOSE PACK,18,24,39
998,"2017 BELLAND, CHASSAGNE-MONTRACHET 1ER MORGEOT 'CLOS PITOIS' BLANC 375ML",366151 |10 BARREL APOCALYPSE IPA 4/6/12OZ BOTTLE,23,22,31


In [116]:
df_compare['partial'] = df_compare['concat'].apply(partial)

In [109]:
# res = df_compare['partial'].groupby(df_compare['INVOICE_PACKAGE_DESCRIPTION'], group_keys=False).nlargest(3)
res = df_compare.sort_values('partial',ascending=False).groupby('INVOICE_PACKAGE_DESCRIPTION').head(10)

In [110]:
end = time.time()
print("TIme taken for 5 against all:" , (end-start))

TIme taken for 5 against all: 1.6494574546813965


In [ ]:
res.sort_values('INVOICE_PACKAGE_DESCRIPTION')

In [ ]:
export_file = res[['INVOICE_PACKAGE_DESCRIPTION','joined','partial']]

In [ ]:
filename = export_file['INVOICE_PACKAGE_DESCRIPTION'].head(1).item()
filename = filename.strip()


In [ ]:
export_file.to_csv('~/Downloads/fuzzymatch_of_invoice_prdtdesc/{}.csv'.format(filename))

In [ ]:
# file1 = open("a.csv", "a") 
# file1.write(export_file.to_string())
# file1.close()

In [ ]:
# #tuple example
# tup = ('SAMURAI SOUR HIBISCUS CORIANDER SEA SALT CIDER 250ML','122436 | CBA EXPLORERS VARIETY PACK 2/12 LNNR')
# x0 = jellyfish.soundex(tup[0])
# sep = ''
# out = sep.join(tup[1])
# n = out.split('|')

# x1 = jellyfish.soundex(n[1])
# fuzz.partial_ratio(x0,x1)